In [7]:
from pathlib import Path
import pickle

feat_dir = "./features/"
models_dir = "./models/"

load_file = f"{feat_dir}X_train.pkl"
with open(load_file, "rb") as load_file:
    X_train = pickle.load(load_file)
    
load_file = f"{feat_dir}y_train.pkl"
with open(load_file, "rb") as load_file:
    y_train = pickle.load(load_file)
    
load_file = f"{feat_dir}X_val.pkl"
with open(load_file, "rb") as load_file:
    X_val = pickle.load(load_file)
    
load_file = f"{feat_dir}y_val.pkl"
with open(load_file, "rb") as load_file:
    y_val = pickle.load(load_file)
    
load_file = f"{feat_dir}X_unbal.pkl"
with open(load_file, "rb") as load_file:
    X_unbal = pickle.load(load_file)
    
load_file = f"{feat_dir}y_unbal.pkl"
with open(load_file, "rb") as load_file:
    y_unbal = pickle.load(load_file)

In [8]:
load_file = f"{feat_dir}multiLabelBinarizer.pkl"
with open(load_file, "rb") as load_file:
    mlb = pickle.load(load_file)
    
#Load classes names
with open(f"{feat_dir}classes_dict.pkl", "rb") as classes_file:
    classes_dict = pickle.load(classes_file)

In [9]:
import numpy as np

X = np.concatenate((X_train, X_unbal))
y = np.concatenate((y_train, y_unbal))

In [12]:
X[0].shape

(10, 128)

In [13]:
import keras
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Input, BatchNormalization, Activation, Lambda
import keras.backend as K
import tensorflow as tf

def auroc(y_true, y_pred):
    return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)

def attention_pooling(inputs, **kwargs):
    [out, att] = inputs

    epsilon = 1e-7
    att = K.clip(att, epsilon, 1. - epsilon)
    normalized_att = att / K.sum(att, axis=1)[:, None, :]

    return K.sum(out * normalized_att, axis=1)

def pooling_shape(input_shape):

    if isinstance(input_shape, list):
        (sample_num, time_steps, freq_bins) = input_shape[0]

    else:
        (sample_num, time_steps, freq_bins) = input_shape

    return (sample_num, freq_bins)

time_steps = 10
freq_bins = 128
classes_num = 10

# Hyper parameters
hidden_units = 1024
drop_rate = 0.5
batch_size = 500

# Embedded layers
input_layer = Input(shape=X[0].shape)

a1 = Dense(hidden_units)(input_layer)
a1 = BatchNormalization()(a1)
a1 = Activation('relu')(a1)
a1 = Dropout(drop_rate)(a1)

a2 = Dense(hidden_units)(a1)
a2 = BatchNormalization()(a2)
a2 = Activation('relu')(a2)
a2 = Dropout(drop_rate)(a2)

a3 = Dense(hidden_units)(a2)
a3 = BatchNormalization()(a3)
a3 = Activation('relu')(a3)
a3 = Dropout(drop_rate)(a3)
cla = Dense(hidden_units, activation='linear')(a3)
att = Dense(hidden_units, activation='sigmoid')(a3)

b1 = Lambda(attention_pooling, output_shape=pooling_shape)([cla, att])
b1 = BatchNormalization()(b1)
b1 = Activation(activation='relu')(b1)
b1 = Dropout(drop_rate)(b1)

output_layer = Dense(classes_num, activation='sigmoid')(b1)

model5 = keras.Model(inputs=input_layer, outputs=output_layer, name="qiuqiangkong")
# model5.compile(loss='binary_crossentropy', optimizer="Adam", metrics=["accuracy", auroc])
model5.compile(loss='binary_crossentropy', optimizer="Adam", metrics=["accuracy"])
model5.summary()

Model: "qiuqiangkong"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10, 128)]    0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 10, 1024)     132096      ['input_3[0][0]']                
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 10, 1024)    4096        ['dense_12[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_8 (Activation)      (None, 10, 1024)     0           ['batch_normalization_

In [15]:
import tensorflow as tf

model5.load_weights(f"{models_dir}qiuqiangkong_b64_weights.tf")